In [1]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import dlib
import cv2
import os
protoPath =  "./face_detection_model/deploy.prototxt"
modelPath = "./face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"

In [2]:
#detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
detector = dlib.get_frontal_face_detector()

In [3]:
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")



In [4]:
imagePaths = list(paths.list_images("./dataset"))
 
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []
 
# initialize the total number of faces processed
total = 0

In [5]:
print(imagePaths)

['./dataset/trisha/00005.png', './dataset/trisha/00003.png', './dataset/trisha/00001.png', './dataset/trisha/00002.png', './dataset/trisha/00000.png', './dataset/trisha/00004.png', './dataset/adrian/00003.jpg', './dataset/adrian/00001.png', './dataset/adrian/00004.jpg', './dataset/adrian/00000.png', './dataset/adrian/00005.jpg', './dataset/adrian/00002.jpg', './dataset/unknown/claire_dearing.jpg', './dataset/unknown/ellie_sattler.jpg', './dataset/unknown/john_hammond.jpg', './dataset/unknown/own_grady.jpg', './dataset/unknown/alan_grant.jpg', './dataset/unknown/ian_malcolm.jpg']


In [6]:
margin = 0.2
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# extract the person name from the image path
	print("[INFO] processing image {}/{}".format(i + 1,
		len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]

	# load the image, resize it to have a width of 600 pixels (while
	# maintaining the aspect ratio), and then grab the image
	# dimensions
	image = cv2.imread(imagePath)
	#image = imutils.resize(image, width=600)
	input_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	img_h, img_w, _ = np.shape(input_img)
	#(img_h, img_w) = image.shape[:2]
	#img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

	detected = detector(image, 1)
	if len(detected) > 0:
		for i, d in enumerate(detected):
			x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
			xw1 = max(int(x1 - margin * w), 0)
			yw1 = max(int(y1 - margin * h), 0)
			xw2 = min(int(x2 + margin * w), img_w - 1)
			yw2 = min(int(y2 + margin * h), img_h - 1)
		face =  image[yw1:yw2 + 1, xw1:xw2 + 1, :]
	cv2.imshow('dlib',face)
cv2.destroyAllWindows()    
    
# # 	# construct a blob from the image
# # 	imageBlob = cv2.dnn.blobFromImage(
# # 		cv2.resize(image, (300, 300)), 1.0, (300, 300),
# # 		(104.0, 177.0, 123.0), swapRB=False, crop=False)

# # 	# apply OpenCV's deep learning-based face detector to localize
# # 	# faces in the input image
# # 	detector.setInput(imageBlob)
# # 	detections = detector.forward()

# # 	# ensure at least one face was found
# # 	if len(detections) > 0:
# # 		# we're making the assumption that each image has only ONE
# # 		# face, so find the bounding box with the largest probability
# # 		i = np.argmax(detections[0, 0, :, 2])
# # 		confidence = detections[0, 0, i, 2]

# # 		# ensure that the detection with the largest probability also
# # 		# means our minimum probability test (thus helping filter out
# # 		# weak detections)
# # 		if confidence > 0.5:
# # 			# compute the (x, y)-coordinates of the bounding box for
# # 			# the face
# # 			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
# # 			(startX, startY, endX, endY) = box.astype("int")

# # 			# extract the face ROI and grab the ROI dimensions
# # 			face = image[startY:endY, startX:endX]
# 			(fH, fW) = face.shape[:2]

# 		# ensure the face width and height are sufficiently large
# 			if fW < 20 or fH < 20:
# 				continue

# 			# construct a blob for the face ROI, then pass the blob
# 			# through our face embedding model to obtain the 128-d
# 			# quantification of the face
# 			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
# 				(96, 96), (0, 0, 0), swapRB=True, crop=False)
# 			embedder.setInput(faceBlob)
# 			vec = embedder.forward()

# 			# add the name of the person + corresponding face
# 			# embedding to their respective lists
# 			knownNames.append(name)
# 			knownEmbeddings.append(vec.flatten())
# 	#cv2.imshow("Face Detector", face)


[INFO] processing image 1/18
[INFO] processing image 2/18
[INFO] processing image 3/18
[INFO] processing image 4/18
[INFO] processing image 5/18
[INFO] processing image 6/18
[INFO] processing image 7/18
[INFO] processing image 8/18
[INFO] processing image 9/18
[INFO] processing image 10/18
[INFO] processing image 11/18
[INFO] processing image 12/18
[INFO] processing image 13/18
[INFO] processing image 14/18
[INFO] processing image 15/18
[INFO] processing image 16/18
[INFO] processing image 17/18
[INFO] processing image 18/18


In [7]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open("./embedd", "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] serializing 0 encodings...


In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import argparse
import pickle

In [13]:
!pip install scikit-learn


In [11]:
# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open("embedd", "rb").read())
 
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])

[INFO] loading face embeddings...
[INFO] encoding labels...


In [12]:
# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)

[INFO] training model...


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [13]:
# write the actual face recognition model to disk
f = open("recognize", "wb")
f.write(pickle.dumps(recognizer))
f.close()
 
# write the label encoder to disk
f = open("lee", "wb")
f.write(pickle.dumps(le))
f.close()

In [14]:
 
# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("recognize", "rb").read())
le = pickle.loads(open("lee", "rb").read())

In [18]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
image = cv2.imread("./images/patrick_bateman.jpg")
image = imutils.resize(image, width=600)
#(h, w) = image.shape[:2]
(img_h, img_w) = image.shape[:2]
	#img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

detected = detector(image, 1)
if len(detected) > 0:
	for i, d in enumerate(detected):
		x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
		xw1 = max(int(x1 - margin * w), 0)
		yw1 = max(int(y1 - margin * h), 0)
		xw2 = min(int(x2 + margin * w), img_w - 1)
		yw2 = min(int(y2 + margin * h), img_h - 1)
		face =  image[yw1:yw2 + 1, xw1:xw2 + 1, :]
			#cv2.imshow("Face Detector", face) 

		(fH, fW) = face.shape[:2]

		# ensure the face width and height are sufficiently large
		if fW < 20 or fH < 20:
			continue

			# construct a blob for the face ROI, then pass the blob
			# through our face embedding model to obtain the 128-d
			# quantification of the face
		faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
			(96, 96), (0, 0, 0), swapRB=True, crop=False)
		embedder.setInput(faceBlob)
		vec = embedder.forward()

 
		# perform classification to recognize the face
		preds = recognizer.predict_proba(vec)[0]
		j = np.argmax(preds)
		proba = preds[j]
		name = le.classes_[j]

		# draw the bounding box of the face along with the associated
		# probability
		text = "{}: {:.2f}%".format(name, proba * 100)
		#y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(image, (xw1, yw1), (xw2, yw2),
			(0, 0, 255), 2)
		cv2.putText(image, text, (xw1, yw1-10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
 
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)

13